In [23]:
import pandas as pd

df=pd.read_json('problems_data.jsonl',lines=True)


In [24]:
df.head()

,title,description,input_description,output_description,sample_io,problem_class,problem_score,url
0,Uuu,Unununium (Uuu) was the name of the chemical\n...,The input consists of one line with two intege...,The output consists of $M$ lines where the $i$...,"[{'input': '7 10', 'output': '1 2 2 3 1 3 3 4 ...",hard,9.7,https://open.kattis.com/problems/uuu
1,House Building,A number of eccentrics from central New York h...,"The input consists of $10$ test cases, which a...",Print $K$ lines with\n the positions of the...,"[{'input': '0 2 3 2 50 60 50 30 50 40', 'outpu...",hard,9.7,https://open.kattis.com/problems/husbygge
2,Mario or Luigi,Mario and Luigi are playing a game where they ...,,,"[{'input': '', 'output': ''}]",hard,9.6,https://open.kattis.com/problems/marioorluigi
3,The Wire Ghost,Žofka is bending a copper wire. She starts wit...,The first line contains two integers $L$ and $...,The output consists of a single line consistin...,"[{'input': '4 3 3 C 2 C 1 C', 'output': 'GHOST...",hard,9.6,https://open.kattis.com/problems/thewireghost
4,Barking Up The Wrong Tree,"Your dog Spot is let loose in the park. Well, ...",The first line of input consists of two intege...,Write a single line containing the length need...,"[{'input': '2 0 10 0 10 10', 'output': '14.14'...",hard,9.6,https://open.kattis.com/problems/barktree


In [25]:
df.describe()

,problem_score
count,4112.000000
mean,5.114689
std,2.177770
min,1.100000
25%,3.300000
50%,5.200000
75%,6.900000
max,9.700000


In [26]:
df['problem_class'].value_counts()

problem_class
hard      1941
medium    1405
easy       766
Name: count, dtype: int64

In [27]:
df.isnull().sum()

title                 0
description           0
input_description     0
output_description    0
sample_io             0
problem_class         0
problem_score         0
url                   0
dtype: int64

In [28]:

grp = df.groupby('problem_class')['problem_score'].agg(['min', 'max'])
grp['range'] = grp['max'] - grp['min']

# show results sorted by range (largest first)
print(grp.sort_values('range', ascending=False))

               min  max  range
problem_class                 
hard           5.5  9.7    4.2
medium         2.8  5.5    2.7
easy           1.1  2.8    1.7


In [29]:
text=['title','description','input_description','output_description','sample_io']

df['only_text']=df[text].astype(str).agg(' '.join, axis=1)
df.head()

,title,description,input_description,output_description,sample_io,problem_class,problem_score,url,only_text
0,Uuu,Unununium (Uuu) was the name of the chemical\n...,The input consists of one line with two intege...,The output consists of $M$ lines where the $i$...,"[{'input': '7 10', 'output': '1 2 2 3 1 3 3 4 ...",hard,9.7,https://open.kattis.com/problems/uuu,Uuu Unununium (Uuu) was the name of the chemic...
1,House Building,A number of eccentrics from central New York h...,"The input consists of $10$ test cases, which a...",Print $K$ lines with\n the positions of the...,"[{'input': '0 2 3 2 50 60 50 30 50 40', 'outpu...",hard,9.7,https://open.kattis.com/problems/husbygge,House Building A number of eccentrics from cen...
2,Mario or Luigi,Mario and Luigi are playing a game where they ...,,,"[{'input': '', 'output': ''}]",hard,9.6,https://open.kattis.com/problems/marioorluigi,Mario or Luigi Mario and Luigi are playing a g...
3,The Wire Ghost,Žofka is bending a copper wire. She starts wit...,The first line contains two integers $L$ and $...,The output consists of a single line consistin...,"[{'input': '4 3 3 C 2 C 1 C', 'output': 'GHOST...",hard,9.6,https://open.kattis.com/problems/thewireghost,The Wire Ghost Žofka is bending a copper wire....
4,Barking Up The Wrong Tree,"Your dog Spot is let loose in the park. Well, ...",The first line of input consists of two intege...,Write a single line containing the length need...,"[{'input': '2 0 10 0 10 10', 'output': '14.14'...",hard,9.6,https://open.kattis.com/problems/barktree,Barking Up The Wrong Tree Your dog Spot is let...


In [30]:
import re
import string
import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text, remove_stopwords=True, min_len=3):
    if pd.isna(text):
        return "", []
    s = str(text)
    # lower
    s = s.lower()
    # remove html tags
    s = re.sub(r'<[^>]+>', ' ', s)
    # remove urls and emails
    s = re.sub(r'http\S+|www\.\S+|\S+@\S+', ' ', s)
    # replace punctuation with spaces (keep internal apostrophes if desired)
    s = s.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation)))
    # remove digits (optional)
    s = re.sub(r'\d+', ' ', s)
    # collapse whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    if not s:
        return "", []
    # tokenize
    tokens = word_tokenize(s)
    # lemmatize & filter
    cleaned = []
    for t in tokens:
        t = t.strip()
        if not t:
            continue
        if remove_stopwords and t in stop_words:
            continue
        if len(t) < min_len:
            continue
        t = lemmatizer.lemmatize(t)
        cleaned.append(t)
    return " ".join(cleaned), cleaned

res = df['only_text'].apply(lambda t: preprocess_text(t, remove_stopwords=True))
df['clean_text'] = res.apply(lambda x: x[0])



df[['only_text', 'clean_text']].head()


,only_text,clean_text
0,Uuu Unununium (Uuu) was the name of the chemic...,uuu unununium uuu name chemical element atom n...
1,House Building A number of eccentrics from cen...,house building number eccentric central new yo...
2,Mario or Luigi Mario and Luigi are playing a g...,mario luigi mario luigi playing game pick dist...
3,The Wire Ghost Žofka is bending a copper wire....,wire ghost žofka bending copper wire start str...
4,Barking Up The Wrong Tree Your dog Spot is let...,barking wrong tree dog spot let loose park wel...


In [31]:

candidates = ['only_text', 'clean_text']
source_col = next((c for c in candidates if c in df.columns), None)
if source_col is None:
    raise KeyError(f"No source text column found. Candidates: {candidates}; DataFrame columns: {df.columns.tolist()}")

math_sym_pattern = re.compile(r'[+\-*/=<>^_%∑√π×÷±∫∆≠∞°·\[\]\(\)\{\}\\|]')  # common math chars
latex_cmd_pattern = re.compile(r'\\[a-zA-Z]+')  # \sum, \int, etc.

def extract_metrics(text):
    s = '' if pd.isna(text) else str(text)
    text_length = len(s)                                    # characters
    word_count = len(re.findall(r'\b\w+\b', s))             # words (simple)
    math_chars = len(math_sym_pattern.findall(s))           # individual math chars
    latex_cmds = len(latex_cmd_pattern.findall(s))         # LaTeX commands
    math_sym_count = math_chars + latex_cmds

    graph_count = len(re.findall(r'\bgraph\b', s, flags=re.I))
    dp_count = len(re.findall(r'\bdynamic\s+programming\b', s, flags=re.I))
    recursion_count = len(re.findall(r'\brecursion\b', s, flags=re.I))

    key_terms_count = graph_count + dp_count + recursion_count

    return pd.Series({
        'text_length': text_length,
        'word_count': word_count,
        'math_sym_count': math_sym_count,
        'graph_count': graph_count,
        'dp_count': dp_count,
        'recursion_count': recursion_count,
        'key_terms_count': key_terms_count
    })

metrics = df[source_col].apply(extract_metrics)
df = pd.concat([df, metrics], axis=1)


df[[source_col, 'text_length', 'word_count', 'math_sym_count',
    'graph_count', 'dp_count', 'recursion_count', 'key_terms_count']].head()


,only_text,text_length,word_count,math_sym_count,graph_count,dp_count,recursion_count,key_terms_count
0,Uuu Unununium (Uuu) was the name of the chemic...,1754,310,45,5,0,0,5
1,House Building A number of eccentrics from cen...,1488,259,43,0,0,0,0
2,Mario or Luigi Mario and Luigi are playing a g...,1364,229,30,0,0,0,0
3,The Wire Ghost Žofka is bending a copper wire....,1494,259,45,0,0,0,0
4,Barking Up The Wrong Tree Your dog Spot is let...,2367,433,30,0,0,0,0


In [32]:
df['key_terms_count'].describe()

count    4112.000000
mean        0.090467
std         0.656600
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max        11.000000
Name: key_terms_count, dtype: float64

In [33]:
df['math_sym_count'].describe()

count    4112.000000
mean       54.498054
std        48.162338
min         2.000000
25%        29.000000
50%        45.000000
75%        68.000000
max      1327.000000
Name: math_sym_count, dtype: float64

In [34]:
colstofilter=['clean_text','math_sym_count','text_length','word_count','problem_class','problem_score']

filtereddf=df[colstofilter].copy()


In [35]:
filtereddf.head()

,clean_text,math_sym_count,text_length,word_count,problem_class,problem_score
0,uuu unununium uuu name chemical element atom n...,45,1754,310,hard,9.7
1,house building number eccentric central new yo...,43,1488,259,hard,9.7
2,mario luigi mario luigi playing game pick dist...,30,1364,229,hard,9.6
3,wire ghost žofka bending copper wire start str...,45,1494,259,hard,9.6
4,barking wrong tree dog spot let loose park wel...,30,2367,433,hard,9.6


In [36]:
from sklearn.model_selection import train_test_split

X=filtereddf.drop(columns=['problem_score','problem_class'])
y_r=filtereddf['problem_score']
y_class=filtereddf['problem_class']

X_train, X_test, yr_train, yr_test, yc_train, yc_test = train_test_split(
    X, y_r, y_class, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_class
)

In [37]:
X_test

,clean_text,math_sym_count,text_length,word_count
1335,investigating imposter stumbled upon village v...,36,1285,214
2182,multiplication game alice bob class drill mult...,67,1207,219
3717,convex polygon area problem need able calculat...,26,841,154
3501,another brick wall construction worker previou...,38,1786,317
3199,waterfall level designer super mawio bros crea...,95,3175,474
...,...,...,...,...
288,pizza cutting pizza usually cooked circle ofte...,52,1770,312
235,arable area prime minister recently bought pie...,59,1503,265
2493,using digit must begin stepping onto square cl...,68,2541,449
2138,vin diagram venn diagram invented great logici...,65,3396,566


In [38]:
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler


text_col = 'clean_text' 
num_cols = [c for c in X.columns if c != text_col]

# Define the transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('text', TfidfVectorizer(max_features=1000), text_col),
        ('num', StandardScaler(), num_cols) 
    ]
)

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

In [39]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize and Train
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train_transformed, yr_train)

# Evaluate
yr_pred = regressor.predict(X_test_transformed)
print(f"Regressor R2 Score: {r2_score(yr_test, yr_pred):.4f}")
print(f"Regressor RMSE: {mean_squared_error(yr_test, yr_pred):.4f}")

Regressor R2 Score: 0.1548
Regressor RMSE: 4.1017


In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Initialize and Train
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train_transformed, yc_train)

# Evaluate
yc_pred = classifier.predict(X_test_transformed)
print(f"Classifier Accuracy: {accuracy_score(yc_test, yc_pred):.4f}")
print("\nClassification Report:")
print(classification_report(yc_test, yc_pred))

Classifier Accuracy: 0.5140

Classification Report:
              precision    recall  f1-score   support

        easy       0.56      0.29      0.38       153
        hard       0.54      0.80      0.64       389
      medium       0.41      0.25      0.31       281

    accuracy                           0.51       823
   macro avg       0.50      0.44      0.44       823
weighted avg       0.50      0.51      0.48       823



In [41]:
from sklearn.preprocessing import LabelEncoder

# Encode classification labels (e.g., 'Easy' -> 0, 'Hard' -> 1)
le = LabelEncoder()
yc_train_encoded = le.fit_transform(yc_train)
yc_test_encoded = le.transform(yc_test)

In [42]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score

xgb_reg = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42
)

# Train
xgb_reg.fit(X_train_transformed, yr_train)

# Evaluate
yr_pred = xgb_reg.predict(X_test_transformed)
print(f"XGBoost Regressor R2: {r2_score(yr_test, yr_pred):.4f}")
print(f"XGBoost Regressor RMSE: {mean_squared_error(yr_test, yr_pred):.4f}")

XGBoost Regressor R2: 0.1565
XGBoost Regressor RMSE: 4.0934


In [43]:

xgb_clf = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    use_label_encoder=False,
    random_state=42
)

# Train using the ENCODED labels
xgb_clf.fit(X_train_transformed, yc_train_encoded)

# Evaluate
yc_pred_encoded = xgb_clf.predict(X_test_transformed)
yc_pred = le.inverse_transform(yc_pred_encoded) # Convert back to original labels

print(f"XGBoost Classifier Accuracy: {accuracy_score(yc_test, yc_pred):.4f}")
print(classification_report(yc_test, yc_pred))

c:\Users\ANSUMAN SAHU\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:27:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Classifier Accuracy: 0.5091
              precision    recall  f1-score   support

        easy       0.49      0.31      0.38       153
        hard       0.56      0.76      0.65       389
      medium       0.38      0.27      0.31       281

    accuracy                           0.51       823
   macro avg       0.48      0.45      0.45       823
weighted avg       0.49      0.51      0.48       823



In [44]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

def map_score_to_class(score):
    if 1.1 <= score < 2.8:
        return 'easy'
    elif 2.8 <= score < 5.5:
        return 'medium'
    elif 5.5 <= score <= 9.7:
        return 'hard'
    else:
        
        if score < 1.1: return 'easy'
        return 'hard'

yr_pred = regressor.predict(X_test_transformed)

yc_pred_from_reg = [map_score_to_class(s) for s in yr_pred]

In [45]:

yc_test_cleaned = yc_test.str.lower().str.strip()

print("--- Classification Report (Derived from Regression Output) ---")
print(f"Accuracy: {accuracy_score(yc_test_cleaned, yc_pred_from_reg):.4f}")
print("\nDetailed Report:")
print(classification_report(yc_test_cleaned, yc_pred_from_reg))

--- Classification Report (Derived from Regression Output) ---
Accuracy: 0.4520

Detailed Report:
              precision    recall  f1-score   support

        easy       0.67      0.04      0.07       153
        hard       0.64      0.42      0.51       389
      medium       0.36      0.72      0.48       281

    accuracy                           0.45       823
   macro avg       0.56      0.39      0.35       823
weighted avg       0.55      0.45      0.42       823



In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import joblib


reg_models = {
    'RandomForest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'n_estimators': [50, 100],
            'max_depth': [None, 10, 20]
        }
    },
    'XGBoost': {
        'model': xgb.XGBRegressor(random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'learning_rate': [0.01, 0.1],
            'max_depth': [4, 6]
        }
    }
}

best_reg_model = None
best_reg_score = -float('inf')

for name, config in reg_models.items():
    print(f"Tuning {name} Regressor...")
    grid = GridSearchCV(config['model'], config['params'], cv=3, scoring='r2', n_jobs=-1)
    grid.fit(X_train_transformed, yr_train)
    
    print(f"Best R2 for {name}: {grid.best_score_:.4f}")
    
    if grid.best_score_ > best_reg_score:
        best_reg_score = grid.best_score_
        best_reg_model = grid.best_estimator_

# Save the best Regressor
joblib.dump(best_reg_model, 'best_problem_regressor.pkl')
print(f"Saved Best Regressor: {type(best_reg_model).__name__}")

Tuning RandomForest Regressor...
Best R2 for RandomForest: 0.1321
Tuning XGBoost Regressor...
Best R2 for XGBoost: 0.1179
Saved Best Regressor: RandomForestRegressor


In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

clf_models = {
    'RandomForest': {
        'model': RandomForestClassifier(random_state=42),
        'params': {
            'n_estimators': [50, 100],
            'criterion': ['gini', 'entropy']
        }
    },
    'XGBoost': {
        'model': xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
        'params': {
            'n_estimators': [100, 200],
            'max_depth': [4, 6],
            'learning_rate': [0.1]
        }
    }
}

best_clf_model = None
best_clf_score = -float('inf')

for name, config in clf_models.items():
    print(f"Tuning {name} Classifier...")
    # Using 'yc_train_encoded' from earlier step
    grid = GridSearchCV(config['model'], config['params'], cv=3, scoring='accuracy', n_jobs=-1)
    grid.fit(X_train_transformed, yc_train_encoded)
    
    print(f"Best Accuracy for {name}: {grid.best_score_:.4f}")
    
    if grid.best_score_ > best_clf_score:
        best_clf_score = grid.best_score_
        best_clf_model = grid.best_estimator_

# Save the best Classifier
joblib.dump(best_clf_model, 'best_problem_classifier.pkl')
print(f"Saved Best Classifier: {type(best_clf_model).__name__}")

Tuning RandomForest Classifier...
Best Accuracy for RandomForest: 0.5081
Tuning XGBoost Classifier...


c:\Users\ANSUMAN SAHU\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [22:44:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Best Accuracy for XGBoost: 0.5056
Saved Best Classifier: RandomForestClassifier


In [48]:
joblib.dump(preprocessor, 'preprocessor.pkl')

['preprocessor.pkl']

In [49]:
joblib.dump(le, 'label_encoder.pkl')

['label_encoder.pkl']